In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import autokeras as ak
import os 
import tensorflow as tf

In [2]:
data = np.load('/work/n1000000_0804_all_flat.npz')
cut_index = np.load('/work/NIO_largerthan1000_index.npy')

In [3]:
input_state = 2

target = data['delta']
if input_state == 0:
    data_all = data['ve_dune']
elif input_state == 1:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune']])
elif input_state == 2:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune'], data['ve_t2hk'], data['vu_t2hk'], data['vebar_t2hk'], data['vubar_t2hk']])

x_train = data_all[:10000, cut_index]
y_train = target[:10000]/360
x_train2 = data_all[10000:900000, cut_index]
y_train2 = target[10000:900000]/360
x_test = data_all[900000:, cut_index]
y_test = target[900000:]/360

In [6]:
generate = 10
num_of_bins = len(x_train[0])
scale = 0.00001

x_train_gen = np.reshape(np.random.normal(x_train, np.sqrt(x_train)*scale, size = (generate, len(x_train), num_of_bins)), (generate*len(x_train), num_of_bins))
y_train_gen = np.repeat(y_train, generate)
x_train2_gen = np.reshape(np.random.normal(x_train2, np.sqrt(x_train2)*scale, size = (generate, len(x_train2), num_of_bins)), (generate*len(x_train2), num_of_bins))
y_train2_gen = np.repeat(y_train2, generate)
x_test_gen = np.reshape(np.random.normal(x_test, np.sqrt(x_test)*scale, size = (generate, len(x_test), num_of_bins)), (generate*len(x_test), num_of_bins))
y_test_gen = np.repeat(y_test, generate)

In [7]:
clf = ak.StructuredDataRegressor(overwrite=True, max_trials=3)
clf.fit(x_train_gen, y_train_gen,
           validation_split = 0.1,
           batch_size=64,
           epochs=2,
           verbose=1,
           shuffle = True
       )

Trial 3 Complete [00h 00m 10s]
val_loss: 0.09052374958992004

Best val_loss So Far: 0.08199122548103333
Total elapsed time: 00h 00m 33s
INFO:tensorflow:Oracle triggered exit

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/2
1563/1563 [==============================] - 4s 3ms/step - loss: 0.1072 - mean_squared_error: 0.1072
Epoch 2/2
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0845 - mean_squared_error: 0.0845
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatica

In [ ]:
model = clf.export_model()
model.summary()

In [ ]:
model.fit(x_train2_gen, y_train2_gen,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
         )

In [ ]:
model.evaluate(x_test_gen, y_test_gen)
pre_test = model.predict(x_test_gen)

In [ ]:
plt.hist(pre_test, histtype = 'step', bins = 100, label = 'prediction')
plt.hist(y_test, histtype = 'step', bins = 100, label = 'target')
plt.xlabel('delta')
plt.ylabel('number')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(7,7))
plt.plot(pre_test, y_test, '.', alpha = 0.05)
plt.xlabel('prediction')
plt.ylabel('target')
plt.title('delta')

In [ ]:
if input_state == 0:
    modelname = '/work/Regression/models_ve_dune/0803_delta1.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)
elif input_state == 1:
    modelname = '/work/Regression/models_all_dune/0803_delta1.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)
elif input_state == 2:
    modelname = '/work/ML4NO/ML/Regression/perturb_cut/0804_cut1000_delta1.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)